In [1]:
import tensorflow as tf

In [2]:
(x_train,y_train),(x_test,y_test)=tf.keras.datasets.mnist.load_data()

In [3]:
#setting varialbles to be changed easily
inp_width=28
inp_height=28
channels=1
inp_pixels=784
#units in convultional layer
n_conv1=32
n_conv2=64
#filter size
conv1_k=5
conv2_k=5
stride1=1
stride2=1
#pool size
max_pool1_k=2
max_pool2_k=2

n_hidden=1024
n_output=10

inp_size_to_hidden=(inp_width//(max_pool1_k*max_pool2_k))*(inp_height//(max_pool1_k*max_pool2_k))*n_conv2

In [4]:
#weights and baises
weights={
    "wc1":tf.Variable(tf.random_normal_initializer()(shape=[conv1_k,conv1_k,channels,n_conv1])),
    "wc2":tf.Variable(tf.random_normal_initializer()(shape=[conv2_k,conv2_k,n_conv1,n_conv2])),
    "wh1":tf.Variable(tf.random_normal_initializer()(shape=[inp_size_to_hidden,n_hidden])),
    "wo":tf.Variable(tf.random_normal_initializer()(shape=[n_hidden,n_output]))
}

biases={
    "bc1":tf.Variable(tf.random_normal_initializer()(shape=[n_conv1])),
    "bc2":tf.Variable(tf.random_normal_initializer()(shape=[n_conv2])),
    "bh1":tf.Variable(tf.random_normal_initializer()(shape=[n_hidden])),
    "bo":tf.Variable(tf.random_normal_initializer()(shape=[n_output]))
}

In [5]:
def conv(x,weight,bias,stride=1):
    #apply the filter on images
    out=tf.nn.conv2d(x,weight,padding="SAME",strides=[1,stride,stride,1])
    out=tf.nn.bias_add(out,bias)
    out=tf.nn.relu(out)
    return out
def maxpooling(x,k=2):
    return tf.nn.max_pool(x,padding='SAME',ksize=[1,k,k,1],strides=[1,k,k,1])

In [6]:
def cnn(x,weights,biases):
    #first reshape the input
    x=tf.reshape(x,shape=[-1,inp_height,inp_width,channels])
    conv1=conv(x,weights['wc1'],biases['bc1'],stride1)
    conv1_pool=maxpooling(conv1,max_pool1_k)
    conv2=conv(conv1_pool,weights['wc2'],biases['bc2'],stride2)
    conv2_pool=maxpooling(conv2,max_pool2_k)
    hidden_inp=tf.reshape(conv2_pool,shape=[-1,inp_size_to_hidden])
    hidden_out_before_activation=tf.add(tf.matmul(hidden_inp,weights['wh1']),biases['bh1'])
    hidden_output=tf.nn.relu(hidden_out_before_activation)
    
    output=tf.add(tf.matmul(hidden_output,weights['wo']),biases['bo'])
    return output

In [7]:
x = tf.keras.Input(shape=(inp_pixels,), dtype=tf.float32)
y = tf.keras.Input(shape=(n_output,), dtype=tf.int32)
pred=cnn(x,weights,biases)

In [29]:
cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred,labels=y))

In [33]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)
optimize = optimizer.minimize(cost)

TypeError: OptimizerV2.minimize() missing 1 required positional argument: 'var_list'